In [1]:
import os
from subprocess import call
from glob import glob
import shutil

In [3]:
# Cells we want to copy as custom macros
snip_cells = ["sky130_fd_sc_hd__and2_1", "sky130_fd_sc_hd__fa_1", "sky130_fd_sc_hd__edfxtp_1"]
std_cell_prefix = "tt_export_"

# Location of liberty files
PDK_ROOT = os.environ.get('PDK_ROOT')
TECH="sky130_fd_sc_hd"
lib_regex = f"{PDK_ROOT}/sky130A/libs.ref/sky130_fd_sc_hd/lib/*.lib" 
lib_files = glob(lib_regex)

In [8]:
# Build a Klayout script that reads in a GDS, removes non-target cells and prefixes target cells

# Get EVs
OPENLANE_ROOT=os.getenv('OPENLANE_ROOT')
PDK_ROOT=os.getenv('PDK_ROOT')
USER=os.getenv('USER')
UID=os.geteuid()
GID=os.getegid()
CWD=os.getcwd()
LIB_PDK="sky130A/libs.ref"
TECH="sky130_fd_sc_hd"
# Tech path
TECH_PATH=f"{PDK_ROOT}/{LIB_PDK}/{TECH}"

# Construct docker klayout command
klayout_cmd_run=[]
klayout_cmd_run.append('docker')
klayout_cmd_run.append('run')
klayout_cmd_run.append(f'-v {OPENLANE_ROOT}:/openLANE_flow')
klayout_cmd_run.append(f'-v {PDK_ROOT}:{PDK_ROOT}')
klayout_cmd_run.append(f'-v {CWD}:/project') 
klayout_cmd_run.append(f'-v {CWD}/gds:/project/gds') 
klayout_cmd_run.append(f'-e PDK_ROOT={PDK_ROOT}')
klayout_cmd_run.append(f'-u {UID}:{GID}')
klayout_cmd_run.append(f'efabless/openlane')

kl_cmd=f'klayout -b -r /project/{kl_script_name} -rd input={f_gds} -rd output_dir=/project/gds/ -l {PDK_ROOT}/sky130A/libs.tech/klayout/tech/sky130A.lyp -nn {PDK_ROOT}/sky130A/libs.tech/klayout/tech/sky130A.lyt'
with open("./docker_output.log", "w") as output:
    call(" ".join(klayout_cmd_run + [kl_cmd]), shell=True, stdout=output, stderr=output)
print("Docker running command:\n" + " ".join(klayout_cmd_run + [kl_cmd]))

# Command to check GDS afterward:
# docker run -it -v $OPENLANE_ROOT:/openLANE_flow -v $PDK_ROOT:$PDK_ROOT -v $PWD/gds:/project -e PDK_ROOT=$PDK_ROOT -v /tmp/.X11-unix:/tmp/.X11-unix -e DISPLAY=$DISPLAY -e QT_GRAPHICSSYSTEM="native" -u $(id -u $USER):$(id -g $USER) efabless/openlane klayout -s /project/tt_export_sky130_fd_sc_hd.gds -l $PDK_ROOT/sky130A/libs.tech/klayout/tech/sky130A.lyp


[sky130_fd_sc_hd__and2_1, sky130_fd_sc_hd__fa_1, sky130_fd_sc_hd__edfxtp_1]
Docker running command:
docker run -v /opt/tt/openlane:/openLANE_flow -v /opt/tt/pdk:/opt/tt/pdk -v /mada/users/tsheaves/Desktop/tt06_hsc_tdc/src/std_cell_macros:/project -v /mada/users/tsheaves/Desktop/tt06_hsc_tdc/src/std_cell_macros/gds:/project/gds -e PDK_ROOT=/opt/tt/pdk -u 7547:1211 efabless/openlane klayout -b -r /project/klayout_script.rb -rd input=/opt/tt/pdk/sky130A/libs.ref/sky130_fd_sc_hd/gds/sky130_fd_sc_hd.gds -rd output_dir=/project/gds/ -l /opt/tt/pdk/sky130A/libs.tech/klayout/tech/sky130A.lyp -nn /opt/tt/pdk/sky130A/libs.tech/klayout/tech/sky130A.lyt
